In [62]:
import requests
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

# Загрузка настроек из файла config.json
with open('config.json') as config_file:
    config = json.load(config_file)

# Аутентификация в Google Sheets API
scope = ['https://www.googleapis.com/auth/spreadsheets']
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
gc = gspread.authorize(credentials)

# Функция для обновления или создания листа в Google Sheets с заголовками столбцов
def update_or_create_sheet(spreadsheet, sheet_title, rows):
    try:
        sheet = spreadsheet.worksheet(sheet_title)
        sheet.clear()
        sheet.resize(rows=len(rows) + 1, cols=len(rows[0]))
        sheet.insert_rows(rows, row=1)
    except gspread.WorksheetNotFound:
        sheet = spreadsheet.add_worksheet(title=sheet_title, rows=len(rows) + 1, cols=len(rows[0]))
        sheet.insert_rows(rows, row=1)

    headers = [
        "Проверка",
        "Опубликован",
        "Пост",
        "Текст",
        "Комментарии",
        "Лайки",
        "Репосты",
        "Просмотры",
        "Total",
        "Viral",
        "Ads",
        "Report",
        "To Group",
        "Unsubscribe"
    ]
    sheet.insert_row(headers, 1)

url = 'https://api.vk.com/method/stats.getPostReach'
params = {
    'access_token': config['token_My'],
    'v': '5.131',
    'extended': 0,
    'filter': 'reach',
}

groups = config['groups']

for group_id in groups:
    print(f"Статистика постов для сообщества с ID {group_id}:")

    wall_get_params = {
        'access_token': config['token_My'],
        'v': '5.131',
        'owner_id': group_id,
        'count': 100,
        'offset': 0,
        'filter': 'all',
    }
    
    min_post = config['min_post']
    max_post = config['max_post']
    interval = config['interval']

    if min_post is not None:
        wall_get_params['count'] = min_post

    if max_post is not None:
        wall_get_params['count'] = max_post

    all_posts = []

    while True:
        response = requests.get('https://api.vk.com/method/wall.get', params=wall_get_params)
        data = response.json()

        if 'response' in data and 'items' in data['response']:
            posts = data['response']['items']
            
            if interval is not None:
                posts = [post for post in posts if post['date'] >= int(datetime.now().timestamp()) - interval * 24 * 60 * 60]

            all_posts.extend(posts)

            if len(posts) < 100:
                break
            else:
                wall_get_params['offset'] += 100
        else:
            print(f"Ошибка при получении списка постов для сообщества {group_id}: {data.get('error', 'Неизвестная ошибка')}")
            break

    params['owner_id'] = group_id

    stats = []

    for i in range(0, len(all_posts), 30):
        post_ids_batch = [str(post['id']) for post in all_posts[i:i+30]]

        params['post_ids'] = ','.join(post_ids_batch)

        response = requests.get(url, params=params)
        data = response.json()

        if 'response' in data:
            post_reach_list = data['response']

            for post, post_reach in zip(all_posts[i:i+30], post_reach_list):
                post_stats = [
                    datetime.now().strftime("%d.%m.%y %H:%M:%S"),
                    datetime.fromtimestamp(post['date']).strftime("%d.%m.%y %H:%M:%S"),
                    f"https://vk.com/wall{post['owner_id']}_{post['id']}",
                    post['text'],  # Текст
                    post['comments']['count'],
                    post['likes']['count'],
                    post['reposts']['count'],
                    post.get('views', {}).get('count', 0),
                    post_reach.get('reach_total', 0),
                    post_reach.get('reach_viral', 0),
                    post_reach.get('reach_ads', 0),
                    post_reach.get('report', 0),
                    post_reach.get('to_group', 0),
                    post_reach.get('unsubscribe', 0)
                ]
                stats.append(post_stats)
        else:
            print(f"Ошибка при получении статистики для постов сообщества {group_id}: {data.get('error', 'Неизвестная ошибка')}")

    if len(stats) == 0:
        print(f"Статистика постов для сообщества с ID {group_id} не найдена.")
        continue

    target_spreadsheet_id = config['target_spreadsheet_id']
    target_spreadsheet = gc.open_by_key(target_spreadsheet_id)
    sheet_title = str(group_id)

    update_or_create_sheet(target_spreadsheet, sheet_title, stats)



Статистика постов для сообщества с ID -6649905:
Статистика постов для сообщества с ID -1623346:
Статистика постов для сообщества с ID -40554507:
Статистика постов для сообщества с ID -40554507 не найдена.
